In [1]:
import numpy as np

features_files = [
    '/kaggle/input/scenicornot/array_immagini224_completo11voti1.npy',
    '/kaggle/input/scenicornot/array_immagini224_completo11voti2.npy',
    '/kaggle/input/scenicornot/array_immagini224_completo11voti3.npy',
    '/kaggle/input/scenicornot/array_immagini224_completo11voti4.npy',
    '/kaggle/input/scenicornot/array_immagini224_completo11voti5.npy',
    #'/kaggle/input/scenicornot/array_immagini224_completo11voti6.npy',
    #'/kaggle/input/scenicornot/array_immagini224_completo11voti7.npy'
    #'/kaggle/input/scenicornot/array_immagini224_completo11voti8.npy'
]
votes_files = [
    '/kaggle/input/scenicornot/array_voti_completo11voti1.npy',
    '/kaggle/input/scenicornot/array_voti_completo11voti2.npy',
    '/kaggle/input/scenicornot/array_voti_completo11voti3.npy',
    '/kaggle/input/scenicornot/array_voti_completo11voti4.npy',
    '/kaggle/input/scenicornot/array_voti_completo11voti5.npy',
    #'/kaggle/input/scenicornot/array_voti_completo11voti6.npy',
    #'/kaggle/input/scenicornot/array_voti_completo11voti7.npy'
    #'/kaggle/input/scenicornot/array_voti_completo11voti8.npy'

]

features = []
votes = []

for features_file, votes_file in zip(features_files, votes_files):
    features_array = np.load(features_file, allow_pickle=True)
    votes_array = np.load(votes_file, allow_pickle=True)
    
    features.append(features_array)
    votes.append(votes_array)
    
    # Elimina gli array singoli
    del features_array, votes_array

# Concatena gli array
features = np.concatenate(features, axis=0)
votes = np.concatenate(votes, axis=0)

In [2]:
# riduzione dei valori di pixel tra 0 e 1
features  = features/255

In [3]:
import tensorflow as tf

# one hot encoding su 3 classi

votes = np.argmax(votes,axis=1)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
votes = tf.one_hot(votes, depth=10)

In [ ]:
votes

In [ ]:
# augmentation

import imgaug.augmenters as iaa
import tensorflow as tf

def augment_data(images, labels):
    augmenter = iaa.Sequential([
        iaa.GaussianBlur(sigma=(0, 1.0)),
        iaa.AdditiveGaussianNoise(scale=(0, 0.05)),
        iaa.Fliplr(0.5)
    ])
    
    augmented_images = list(images) 
    augmented_labels = list(labels)
    
    for i in range(len(images)):
        top_label = tf.argmax(labels[i])
        if top_label > 1:
            augmented_image = augmenter.augment_image(images[i,:,:,:])
            augmented_images.append(augmented_image)
            augmented_labels.append(labels[i])
    return np.array(augmented_images), np.array(augmented_labels)

aug_features, aug_votes = augment_data(features, votes)

In [ ]:
del features

In [ ]:
import matplotlib.pyplot as plt

# Calcola la frequenza delle classi
class_frequencies = np.sum(votes, axis=0)

# Crea l'istogramma delle frequenze
plt.bar(range(1, 4), class_frequencies)

# Aggiungi etichette agli assi
plt.xlabel('Classe')
plt.ylabel('Frequenza')

# Mostra l'istogramma
plt.show()

In [5]:
from sklearn.model_selection import train_test_split

trainval_features, test_features, trainval_target, test_target = train_test_split(features, np.array(votes), test_size=0.2)

# controllo training set
print('TRAINING SET:\t',trainval_features.shape)

# controllo test set
print('TEST SET:\t',test_features.shape)

TRAINING SET:	 (10514, 224, 224, 3)
TEST SET:	 (2629, 224, 224, 3)


In [6]:
del features

In [7]:
# validazione

indices = np.arange(len(trainval_features))

train_features, val_features, train_target, val_target = train_test_split(trainval_features, trainval_target, test_size=0.1)

# controllo training set
print('TRAINING SET:\t',train_features.shape)

# controllo validation set
print('VALIDATION SET:\t',val_features.shape)

TRAINING SET:	 (9462, 224, 224, 3)
VALIDATION SET:	 (1052, 224, 224, 3)


In [8]:
del trainval_features

In [9]:
from tensorflow import keras

import keras
from keras import backend as K
from tensorflow.keras.applications.vgg16 import VGG16

from keras.models import Model
from tensorflow.keras import Model, Input, optimizers
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from keras.layers import concatenate
from keras.utils import plot_model

In [10]:
keras.backend.clear_session()

In [13]:
base_model = VGG16(weights='/kaggle/input/scenicornot-pretrained-weights/vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(224, 224, 3))


In [12]:
#tagliare il modello
# Creare il nuovo modello tagliato
cut_model = Model(inputs=base_model.inputs, outputs=base_model.layers[11].output)

# Caricare i pesi corrispondenti al nuovo modello
cut_weights = base_model.get_weights()[:16]
cut_model.set_weights(cut_weights)

In [14]:
inputs = Input(shape=(224,224,3))

### PRE-TRAINED FEATURE EXTRACTOR

#freeze mobilenet weights
base_model.trainable = False

#add mobnet to the network only in inference mode (SEE LATER)
x = base_model(inputs, training=False)

#un layer di pooling
x = GlobalAveragePooling2D()(x)

#flatten the feature maps
x = Flatten()(x)

#un layer fully connected prima del finale
x = Dense(256, activation='relu')(x)

#output layer with softmax for classification
outputs = Dense(10, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs, name='base_model')

In [16]:
sgd = optimizers.SGD(learning_rate=0.02)

#def my_cross_entropy(y_true, y_pred)

model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

#define some hyperparameters

epochs = 5
batch_size = 128

In [17]:
history = model.fit(
    train_features,
    train_target,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(val_features,val_target),
    verbose=1 #default, show progress bar
)

Epoch 1/5
74/74 [==============================] - 3041s 41s/step - loss: 1.9909 - accuracy: 0.2261 - val_loss: 1.9118 - val_accuracy: 0.2481
Epoch 2/5
74/74 [==============================] - 3003s 41s/step - loss: 1.8812 - accuracy: 0.2576 - val_loss: 1.8762 - val_accuracy: 0.2490
Epoch 3/5
74/74 [==============================] - 3009s 41s/step - loss: 1.8512 - accuracy: 0.2720 - val_loss: 1.8668 - val_accuracy: 0.2548
Epoch 4/5
74/74 [==============================] - 2999s 41s/step - loss: 1.8274 - accuracy: 0.2838 - val_loss: 1.8522 - val_accuracy: 0.2747
Epoch 5/5
74/74 [==============================] - 2998s 41s/step - loss: 1.8138 - accuracy: 0.2815 - val_loss: 1.8194 - val_accuracy: 0.2719
